In [ ]:
##Create "Total" (sum of SBB filter and Retail filter) for AMs and SMs, and Create National Office Total

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql import HiveContext, SQLContext,SparkSession
from pyspark.sql import functions as F
from datetime import date,timedelta
from datetime import datetime
sc = SparkContext(appName = "Combined_Table")
sql_context = HiveContext(sc)
spark = SparkSession(sc)
spark = (SparkSession.builder.enableHiveSupport().getOrCreate())


In [ ]:
spark 
import pandas as pd
import os
import numpy as np
import datetime
import time
import glob
import os
from numpy import nan
from pyspark.sql import functions as F
from pyspark.sql.functions import format_string

pd.set_option('display.width', 150)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
exec(open('/home/sharb24/SBB/thural2/record.py').read())


In [ ]:
############# DEFINE VARIABLES ##############

gnb_tup=('Total GNB Widgets (Actual)', 'GNB Points 1PT','GNB Points 4PTS','GNB Points 10PTS','GNB Points 12PTS','Total GNB Points',
         'Total SBB Net Authorized Volume','SBB Net Authorized LON Volume','SBB Net Authorized LOC Volume','Net SBB Professional Banking',
         'Net Professional Student LOC','Net Franchise Banking Net Volume','Net Professional Banking Net Volume',
         'Net Professional and Franchise Banking')

lei_tup=('LEI',)

ctp_tup=('Contribution to Profit (CTP)',)

tdef_tup=('TDEF Volume',)

total_onetd=('Total Wealth Referrals','AMSB Retail Referrals','TD Securities Direct Trade')
total_be=('Business Credit Protection(BCP) Widgets','Total Merchant Solutions','Total Cash Management Services (including RDC)',
         'Total Credit Cards','Credit Cards Increases')


In [ ]:
## read in the sbb filter data
sbb_filter=spark.sql("select * from anp_cabbtdct1_final.SBB_SC_METRICS_SBB")
sbb_filter0=sbb_filter.toPandas()
sbb_filter0= sbb_filter0.astype({"record_date": int})
sbb_filter0= sbb_filter0.astype({"record_date": str})
sbb_filter1=sbb_filter0[(sbb_filter0.record_date==rec_date)]

## read in the retail filter data
retail_filter=spark.sql("select * from anp_cabbtdct1_final.SBB_SC_METRICS_RETAIL")
retail_filter0=retail_filter.toPandas()
retail_filter0= retail_filter0.astype({"record_date": int})
retail_filter0= retail_filter0.astype({"record_date": str})
retail_filter1=retail_filter0[(retail_filter0.record_date==rec_date)]

## concatenate the sbb and retail dataframes
sbb_retail_all=pd.concat([sbb_filter1,retail_filter1],axis=0)


In [ ]:
sbb_retail_all.metric_name.unique()

In [ ]:
### filter AMSB for later concatenation
sbb_amsb=sbb_retail_all[(sbb_retail_all.level=='AMSB')]

## filter for AM
sbb_am=sbb_retail_all[(sbb_retail_all.level=='AM')]

## filter for SM
sbb_sm=sbb_retail_all[(sbb_retail_all.level=='SM')]


In [ ]:
##############################  GROUP BY AM ##############################
tot_am=amgrpfunc(sbb_am)

##include the LEVEL variable
tot_am['level']='AM'
tot_am['scorecard_filter']='Total'
tot_am['acf2_id']=''
tot_am['employee_name']=tot_am['am_cost_center_full_name']

##CREATE THE AM QUARTER AND YTD VARIABLES
newvarfunc(tot_am)

## replace 0 with NaN so median only counts non-zeros
tot_am=tot_am.replace(0.0, np.nan)

## CREATE THE QTD BENCHMARK ("quarter" is a set variable (current))
## Q1
tot_am_median=tot_am.groupby(['metric_name'], as_index=False)['q1'].median()
##rename median
tot_am_median.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
### merge to dataframe
tot_am_all3=pd.merge(tot_am,tot_am_median,on=['metric_name'],
                   how="left")

## Q2
tot_am_median1=tot_am.groupby(['metric_name'], as_index=False)['q2'].median()
##rename median
tot_am_median1.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
### merge to dataframe
tot_am_all4=pd.merge(tot_am_all3,tot_am_median1,on=['metric_name'],
                   how="left")

## Q3
tot_am_median2=tot_am.groupby(['metric_name'], as_index=False)['q3'].median()
##rename median
tot_am_median2.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
### merge to dataframe
tot_am_all5=pd.merge(tot_am_all4,tot_am_median2,on=['metric_name'],
                   how="left")

## Q4
tot_am_median3=tot_am.groupby(['metric_name'], as_index=False)['q4'].median()
##rename median
tot_am_median3.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
### merge to dataframe
tot_am_all6=pd.merge(tot_am_all5,tot_am_median3,on=['metric_name'],
                   how="left")


## CREATE THE YTD BENCHMARK ("quarter" is a set variable (current))
tot_am_ytdmedian=tot_am.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
tot_am_ytdmedian.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)
### merge to dataframe
tot_am_all7=pd.merge(tot_am_all6,tot_am_ytdmedian,on=['metric_name'],
                   how="left")

## keep relevant vars
tot_am_all8=tot_am_all7[['metric_name','level','acf2_id','employee_name',
               'november','december','january','february','march','april','may','june','july',
               'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
##############################  GROUP BY SM ##############################
tot_sm=smgrpfunc(sbb_sm)

##include the LEVEL variable
tot_sm['level']='SM'
tot_sm['acf2_id']=''
tot_sm['scorecard_filter']='Total'
tot_sm['employee_name']=tot_sm['sm_cost_center_name']
tot_sm['am_cost_center']=''
tot_sm['am_cost_center_name']=''
tot_sm['am_cost_center_full_name']=''

##CREATE THE AM QUARTER AND YTD VARIABLES
newvarfunc(tot_sm)

## replace 0 with NaN so median only counts non-zeros
tot_sm=tot_sm.replace(0.0, np.nan)

## CREATE THE QTD BENCHMARK ("quarter" is a set variable (current))
## Q1
tot_sm_median=tot_sm.groupby(['metric_name'], as_index=False)['q1'].median()
##rename median
tot_sm_median.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
### merge to dataframe
tot_sm_all3=pd.merge(tot_sm,tot_sm_median,on=['metric_name'],
                   how="left")

## Q2
tot_sm_median1=tot_sm.groupby(['metric_name'], as_index=False)['q2'].median()
##rename median
tot_sm_median1.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
### merge to dataframe
tot_sm_all4=pd.merge(tot_sm_all3,tot_sm_median1,on=['metric_name'],
                   how="left")

## Q3
tot_sm_median2=tot_sm.groupby(['metric_name'], as_index=False)['q3'].median()
##rename median
tot_sm_median2.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
### merge to dataframe
tot_sm_all5=pd.merge(tot_sm_all4,tot_sm_median2,on=['metric_name'],
                   how="left")

## Q4
tot_sm_median3=tot_sm.groupby(['metric_name'], as_index=False)['q4'].median()
##rename median
tot_sm_median3.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
### merge to dataframe
tot_sm_all6=pd.merge(tot_sm_all5,tot_sm_median3,on=['metric_name'],
                   how="left")

## CREATE THE YTD BENCHMARK ("quarter" is a set variable (current))
tot_sm_ytdmedian=tot_sm.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
tot_sm_ytdmedian.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)
### merge to dataframe
tot_sm_all7=pd.merge(tot_sm_all6,tot_sm_ytdmedian,on=['metric_name'],
                   how="left")

## keep relevant vars
tot_sm_all8=tot_sm_all7[['metric_name','level','acf2_id','employee_name',
                'november','december','january','february','march','april','may','june','july',
                'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
##############################  GROUP BY NATIONAL OFFICE ##############################

tot_nso=natgrpfunc(tot_sm_all8)

##include the LEVEL variable
tot_nso['level']='National Office'
tot_nso['acf2_id']=''
tot_nso['scorecard_filter']='Total'
tot_nso['employee_name']='National Office'
tot_nso['am_cost_center']=''
tot_nso['am_cost_center_name']=''
tot_nso['am_cost_center_full_name']=''
tot_nso['sm_cost_center']=''
tot_nso['sm_cost_center_name']=''

##CREATE THE AM QUARTER AND YTD VARIABLES
newvarfunc(tot_nso)

## replace 0 with NaN so median only counts non-zeros
tot_nso=tot_nso.replace(0.0, np.nan)

## CREATE THE QTD BENCHMARK ("quarter" is a set variable (current))
## Q1
tot_nso_median=tot_nso.groupby(['metric_name'], as_index=False)['q1'].median()
##rename median
tot_nso_median.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
### merge to dataframe
tot_nso_all3=pd.merge(tot_nso,tot_nso_median,on=['metric_name'],
                   how="left")

## Q2
tot_nso_median1=tot_nso.groupby(['metric_name'], as_index=False)['q2'].median()
##rename median
tot_nso_median1.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
### merge to dataframe
tot_nso_all4=pd.merge(tot_nso_all3,tot_nso_median1,on=['metric_name'],
                   how="left")

## Q3
tot_nso_median2=tot_nso.groupby(['metric_name'], as_index=False)['q3'].median()
##rename median
tot_nso_median2.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
### merge to dataframe
tot_nso_all5=pd.merge(tot_nso_all4,tot_nso_median2,on=['metric_name'],
                   how="left")

## Q4
tot_nso_median3=tot_nso.groupby(['metric_name'], as_index=False)['q4'].median()
##rename median
tot_nso_median3.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
### merge to dataframe
tot_nso_all6=pd.merge(tot_nso_all5,tot_nso_median3,on=['metric_name'],
                   how="left")

## CREATE THE YTD BENCHMARK ("quarter" is a set variable (current))
tot_nso_ytdmedian=tot_nso.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
tot_nso_ytdmedian.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)
### merge to dataframe
tot_nso_all7=pd.merge(tot_nso_all6,tot_nso_ytdmedian,on=['metric_name'],
                   how="left")

tot_nso_all7['record_date']=rec_date

## keep relevant vars
tot_nso_all8=tot_nso_all7[['metric_name','level','acf2_id','employee_name',
                'november','december','january','february','march','april','may','june','july',
                'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
### APPEND ALL THE DATAFRAME FILTERS (including AMSB) -am total, sm total, national total, 
df_tot_all=pd.concat([tot_am_all8,tot_sm_all8, tot_nso_all8,sbb_amsb, sbb_am, sbb_sm],axis=0)


In [ ]:
############################ read in all other dataframes and append #############################
## read in gnb
df1 = spark.table('anp_cabbtdct1_final.SBB_SC_OUTPUT_GNB')
final_gnb0=df1.toPandas()
final_gnb=final_gnb0[(final_gnb0.record_date==rec_date)]

## read in TDEF
df144 = spark.table('anp_cabbtdct1_final.SBB_SC_OUTPUT_TDEF')
final_tdef0=df144.toPandas()
final_tdef=final_tdef0[(final_tdef0.record_date==rec_date)]

## read in Retail Referrals
df140 = spark.table('anp_cabbtdct1_final.SBB_SC_OUTPUT_RETAIL_REF')
final_retailref0=df140.toPandas()
final_retailref=final_retailref0[(final_retailref0.record_date==rec_date)]

## read in Branch Adoption
df145 = spark.table('anp_cabbtdct1_final.SBB_SC_OUTPUT_BRANCH_ADOPTION')
final_branchadopt0=df145.toPandas()
final_branchadopt=final_branchadopt0[(final_branchadopt0.record_date==rec_date)]

## concatenate
df_full=pd.concat([df_tot_all,final_gnb,final_tdef,final_retailref,final_branchadopt],axis=0)
#removed final GNB dataframe
#df_full=pd.concat([df_tot_all,final_tdef,final_retailref,final_branchadopt],axis=0)


In [ ]:
#df_full.metric_name.unique()
#final_retailref[(final_retailref.metric_name=="AMSB Retail Referrals")&(final_retailref.level=="National Office")]

In [ ]:
#df_full.info()

In [ ]:
### create bbs for now; delete later

if any(element in gnb_tup for element in df_full.metric_name):
    test1=pd.DataFrame({'metric_name':['']})
else:
    test1 = pd.DataFrame({'metric_name':gnb_tup})

### create LEI metric anyway  (in case due to no data in Nov; will delete later if data exists)
if any(element in lei_tup for element in df_full.metric_name):
    test2=pd.DataFrame({'metric_name':['']})
else:
    test2 = pd.DataFrame({'metric_name':lei_tup}) 

if any(element in ctp_tup for element in df_full.metric_name):
    test3=pd.DataFrame({'metric_name':['']}) 
else:
    test3 = pd.DataFrame({'metric_name':ctp_tup})

if any(element in tdef_tup for element in df_full.metric_name):
    test4=pd.DataFrame({'metric_name':['']})
else:
    test4 = pd.DataFrame({'metric_name':tdef_tup}) 

if any(element in 'Referrals to BBS' for element in df_full.metric_name):
    test00=pd.DataFrame({'metric_name':['']})
else:
    test00 = pd.DataFrame({'metric_name':['Referrals to BBS']}) 

## concatenate
df_test=pd.concat([test1,test2,test3,test4,test00],axis=0)
df_test['temp']=1

In [ ]:
df_test.metric_name.unique()

In [ ]:
######################## CREATE GNB, LEI METRICS FOR START OF FISCAL (DUE TO NO DATA) #######################################

lev_list=('AMSB','AM','SM','National Office')
sc_list=('SBB','Retail','Total')

## create dfs
df_test3 = pd.DataFrame({'level':lev_list})
df_test3['temp']=1

df_test4 = pd.DataFrame({'scorecard_filter':sc_list})
df_test4['temp']=1

result = pd.merge(pd.merge(df_test,df_test3,on=['temp']),df_test4,on=['temp'])
result = result.drop('temp', axis=1)

## for National Office, drop 'SBB' and 'Retail' scorecard filters; (df contains 'metric_name','level','scorecard_filter')
result1=result.loc[((result.level!='National Office') | (result.scorecard_filter!='SBB')) &
                   ((result.level!='National Office') | (result.scorecard_filter!='Retail'))]

## for AMSB, drop 'Retail' and 'Total' scorecard filters
result2=result1.loc[((result1.level!='AMSB') | (result1.scorecard_filter!='Total')) &
                   ((result1.level!='AMSB') | (result1.scorecard_filter!='Retail'))]

##remove blank metric_name
result2=result2[(result2.metric_name!='')]


In [ ]:
result2.head(4)

In [ ]:
####################### get all AMSBs
df=spark.sql("select acf2_id,employee_name,am_cost_center,am_cost_center_name,am_cost_center_full_name, \
                     sm_cost_center,sm_cost_center_name from \
                     (select *, row_number() over(partition by acf2_id order by record_date desc) as ran \
                     from anp_cabbtdct1_final.HRM_SBB_ALIGNMENT0)b where ran=1")
sbb_hr1=df.toPandas()

sbb_hr2=sbb_hr1.copy()
sbb_hr2['level']='AMSB'
sbb_hr3=sbb_hr2[['acf2_id','employee_name','level','am_cost_center','am_cost_center_name','am_cost_center_full_name',
                 'sm_cost_center','sm_cost_center_name']]

result8=result2[(result2.level=='AMSB')]

## merge HR data with all metric names
result_amsb18 = pd.merge(sbb_hr3,result8,on=['level'],how='left')

## order vars
result_amsb288=result_amsb18[['metric_name','level','scorecard_filter','acf2_id','employee_name','am_cost_center',
                             'am_cost_center_name','am_cost_center_full_name','sm_cost_center','sm_cost_center_name']]

######################## get all AMs
sbb_hr8=sbb_hr1.copy()
sbb_hr8['level']='AM'
sbb_hr9=sbb_hr8[['level','am_cost_center','am_cost_center_name','am_cost_center_full_name','sm_cost_center','sm_cost_center_name']].drop_duplicates()

result10=result2[(result2.level=='AM')]

## merge HR data with all metric names
result_am28 = pd.merge(sbb_hr9,result10,on=['level'],how='left')

result_am28['acf2_id']=''
result_am28['employee_name']=result_am28.am_cost_center_full_name

## order vars
result_am288=result_am28[['metric_name','level','scorecard_filter','acf2_id','employee_name','am_cost_center',
                             'am_cost_center_name','am_cost_center_full_name','sm_cost_center','sm_cost_center_name']]


########################### get all SMs
sbb_hr66=sbb_hr1.copy()
sbb_hr66['level']='SM'
sbb_hr99=sbb_hr66[['level','sm_cost_center','sm_cost_center_name']].drop_duplicates()

result100=result2[(result2.level=='SM')]

## merge HR data with all metric names
result_sm288 = pd.merge(sbb_hr99,result100,on=['level'],how='left')

result_sm288['acf2_id']=''
result_sm288['am_cost_center']=''
result_sm288['am_cost_center_name']=''
result_sm288['am_cost_center_full_name']=''
result_sm288['employee_name']=result_sm288.sm_cost_center_name

## order vars
result_sm2881=result_sm288[['metric_name','level','scorecard_filter','acf2_id','employee_name','am_cost_center',
                             'am_cost_center_name','am_cost_center_full_name','sm_cost_center','sm_cost_center_name']]


########################### get NATIONAL OFFICE

result16=result2[(result2.level=='National Office')]

result16['acf2_id']=''
result16['am_cost_center']=''
result16['am_cost_center_name']=''
result16['am_cost_center_full_name']=''
result16['sm_cost_center']=''
result16['sm_cost_center_name']=''
result16['employee_name']='National Office'

## order vars
result_nso288=result16[['metric_name','level','scorecard_filter','acf2_id','employee_name','am_cost_center',
                             'am_cost_center_name','am_cost_center_full_name','sm_cost_center','sm_cost_center_name']]


########### CONCATENATE amsb, am, sm, national office
df_all9=pd.concat([result_amsb288,result_am288,result_sm2881, result_nso288],axis=0)


In [ ]:
#df_all9.info()

In [ ]:
######### create the columns

cols = ['metric_name','level','scorecard_filter','acf2_id','employee_name','am_cost_center','am_cost_center_name','am_cost_center_full_name',
        'sm_cost_center','sm_cost_center_name','november','december','january','february','march','april','may','june','july',
        'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
        'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark']
net_a = pd.DataFrame(columns=cols)

complementary = [c for c in net_a if c not in df_all9]
new= pd.concat([net_a[complementary], df_all9], axis=1, join='outer')
#new= pd.merge(net_a, df_all9, how='outer')
#new= pd.concat([net_a, df_all9], axis=1, join='outer')


In [ ]:
#new.head(5)

In [ ]:
##delete the extra filler row
#new= new.drop(labels='[\,]', axis=0)
##convert to float
new = new.astype({"november": float,"december": float,"january": float, 
                                    "february": float, "march": float,"april": float,"may": float,"june": float,
                                    "july": float,"august": float,"september": float,"october": float,"q1": float,
                                    "q1_benchmark": float,"q2": float,"q2_benchmark": float,"q3": float,"q3_benchmark": float,
                                    "q4": float,"q4_benchmark": float,"ytd": float,"ytd_benchmark": float})

new['record_date']=rec_date

## order the vars
result_amsb3=new[['metric_name','level','acf2_id','employee_name', 
                            'november','december','january','february','march','april','may','june','july',
                            'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
                            'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
                            'am_cost_center','am_cost_center_name','am_cost_center_full_name',
                            'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
## concatenate the total dataframe with the month-delayed data
df_sc_all=pd.concat([df_full, result_amsb3],axis=0)


In [ ]:
################ FILL BLANKS WITH 0s
df_sc_all[['november','december','january','february','march','april','may','june','july','august','september','october','q1',
           'q1_benchmark','q2','q2_benchmark','q3','q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark']]=df_sc_all[['november','december','january','february','march','april','may','june','july',
                                                                                                                         'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3','q3_benchmark',
                                                                                                                         'q4','q4_benchmark','ytd','ytd_benchmark']].fillna(value=0)


In [ ]:
############################## CREATE ADDITIONAL METRICS ##################################
## select the metrics from gnb net that are the same as gross
gross_list=('Net SBB Professional Banking','Net Franchise Banking Net Volume')
all_month_final2=df_sc_all[df_sc_all['metric_name'].isin(gross_list)]
##rename metric_name to gross
all_month_final2['metric_name'] = all_month_final2['metric_name'].replace(['Net SBB Professional Banking', 'Net Franchise Banking Net Volume'], 
                                                                          ['Gross SBB Professional Banking', 'Gross Franchise Banking Gross Volume'])

## order vars
all_month_final2=all_month_final2[['metric_name','level','acf2_id','employee_name', 
                            'november','december','january','february','march','april','may','june','july',
                            'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
                            'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
                            'am_cost_center','am_cost_center_name','am_cost_center_full_name',
                            'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]

### concatenate
all_month_final3=pd.concat([df_sc_all,all_month_final2],axis=0)


In [ ]:
all_month_final3.metric_name.unique()

In [ ]:
################################# calculate total metrics ###################################

##(1) calculate Gross Professional Banking Gross Volume
gross1_list=('Gross SBB Professional Banking','Gross Professional Student LOC')
all_month_gross=all_month_final3[all_month_final3['metric_name'].isin(gross1_list)]
all_month_gross['metric_name'] = all_month_gross['metric_name'].replace(['Gross SBB Professional Banking','Gross Professional Student LOC'], 
                                                                          'Gross Professional Banking Gross Volume')

##(2) calculate Total Gross Professional and Franchise Banking
tot_gross_list=('Gross SBB Professional Banking','Gross Professional Student LOC','Gross Franchise Banking Gross Volume')
all_month_gross1=all_month_final3[all_month_final3['metric_name'].isin(tot_gross_list)]
all_month_gross1['metric_name'] = all_month_gross1['metric_name'].replace(['Gross SBB Professional Banking','Gross Professional Student LOC','Gross Franchise Banking Gross Volume'], 
                                                                          'Total Gross Professional and Franchise Banking')

##(3) calculate Total Business Banking Referral Volume
tot_ref_list=('ALL Total Commercial Volume','Total MUR Mortgage Volume', 'TDEF Volume')
all_month_gross2=all_month_final3[all_month_final3['metric_name'].isin(tot_ref_list)]
all_month_gross2['metric_name'] = all_month_gross2['metric_name'].replace(['ALL Total Commercial Volume','Total MUR Mortgage Volume','TDEF Volume'], 
                                                                          'Total Business Banking Referral Volume')

## concatenate these dataframes
all_month_final30=pd.concat([all_month_gross2,all_month_gross1,all_month_gross],axis=0)

## order vars
all_month_final31=all_month_final30[['metric_name','level','acf2_id','employee_name', 
                            'november','december','january','february','march','april','may','june','july',
                            'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
                            'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
                            'am_cost_center','am_cost_center_name','am_cost_center_full_name',
                            'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
################# CALCULATE ONETD WIDGETS (SUM OF RETAIL REFERRALS, WEALTH REFERRALS, TD SECURITIES)
################# CALCULATE BUSINESS ESSENTIALS (SUM OF BCP, CREDIT CARDS, MERCHANT SOLUTIONS, CMS, REFERRALS TO BBS)
################# CALCULATE  the other total metrics

######### get AMSB first

df_sc_all0=df_sc_all[(df_sc_all.level=='AMSB')]
df_sc_all1=df_sc_all0[df_sc_all0.metric_name.isin(total_onetd)]
df_sc_all1=df_sc_all1.assign(metric_name='Total OneTD Widgets')

## Business Essentials
df_be_all0=df_sc_all[(df_sc_all.level=='AMSB')]
df_be_all1=df_be_all0[df_be_all0.metric_name.isin(total_be)]
df_be_all1=df_be_all1.assign(metric_name='Total Business Essentials Widgets')

### order the variables and concatenate with other total metrics
df_be_all31=df_be_all1[['metric_name','level','acf2_id','employee_name', 
                            'november','december','january','february','march','april','may','june','july',
                            'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
                            'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
                            'am_cost_center','am_cost_center_name','am_cost_center_full_name',
                            'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]

df_sc_all31=df_sc_all1[['metric_name','level','acf2_id','employee_name', 
                            'november','december','january','february','march','april','may','june','july',
                            'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
                            'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
                            'am_cost_center','am_cost_center_name','am_cost_center_full_name',
                            'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]

## get only the AMSBs from the other total metrics
all_month_final32=all_month_final31[(all_month_final31.level=='AMSB')]

## concatenate all dataframes
all_metrics_all=pd.concat([all_month_final32,df_be_all31,df_sc_all31],axis=0)


##group by
df_sc_all3=all_metrics_all.groupby(by=['level','scorecard_filter','metric_name','acf2_id','employee_name',
                 'am_cost_center','am_cost_center_name','am_cost_center_full_name',
                 'sm_cost_center','sm_cost_center_name','record_date'],as_index=False).aggregate({
                        'november': 'sum',
                        'december': 'sum',
                        'january': 'sum',
                        'february': 'sum',
                        'march': 'sum',
                        'april': 'sum',
                        'may': 'sum',
                        'june': 'sum',
                        'july': 'sum',
                        'august': 'sum',
                        'september': 'sum',
                        'october': 'sum'})
    
##CREATE THE AM QUARTER AND YTD VARIABLES
newvarfunc(df_sc_all3)

## replace 0 with NaN so median only counts non-zeros
df_sc_all3=df_sc_all3.replace(0.0, np.nan)


## CREATE THE QTD BENCHMARK ("quarter" is a set variable (current))
## Q1
psa_sc_median=df_sc_all3.groupby(['metric_name','sm_cost_center'], as_index=False)['q1'].median()
##rename median
psa_sc_median.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
### merge to dataframe
psa_sc_all4=pd.merge(df_sc_all3,psa_sc_median,on=['metric_name','sm_cost_center'],
                   how="left")

## Q2
psa_sc_median1=df_sc_all3.groupby(['metric_name','sm_cost_center'], as_index=False)['q2'].median()
##rename median
psa_sc_median1.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
### merge to dataframe
psa_sc_all4=pd.merge(psa_sc_all4,psa_sc_median1,on=['metric_name','sm_cost_center'],
                   how="left")

## Q3
psa_sc_median2=df_sc_all3.groupby(['metric_name','sm_cost_center'], as_index=False)['q3'].median()
##rename median
psa_sc_median2.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
### merge to dataframe
psa_sc_all5=pd.merge(psa_sc_all4,psa_sc_median2,on=['metric_name','sm_cost_center'],
                   how="left")

## Q4
psa_sc_median3=df_sc_all3.groupby(['metric_name','sm_cost_center'], as_index=False)['q4'].median()
##rename median
psa_sc_median3.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
### merge to dataframe
psa_sc_all6=pd.merge(psa_sc_all5,psa_sc_median3,on=['metric_name','sm_cost_center'],
                   how="left")


## CREATE THE YTD BENCHMARK ("quarter" is a set variable (current))
psa_sc_ytdmedian=df_sc_all3.groupby(['metric_name','sm_cost_center'], as_index=False)['ytd'].median()
##rename median
psa_sc_ytdmedian.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)
### merge to dataframe
psa_sc_all7=pd.merge(psa_sc_all6,psa_sc_ytdmedian,on=['metric_name','sm_cost_center'],
                   how="left")

## keep relevant vars
psa_sc_all8=psa_sc_all7[['metric_name','level','acf2_id','employee_name',
                'november','december','january','february','march','april','may','june','july',
                'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


############### group by AM
## OneTD
df_am_all0=df_sc_all[(df_sc_all.level=='AM')]
df_am_all1=df_am_all0[df_am_all0.metric_name.isin(total_onetd)]
df_am_all1=df_am_all1.assign(metric_name='Total OneTD Widgets')

## BE
df_am1_all0=df_sc_all[(df_sc_all.level=='AM')]
df_am1_all1=df_am1_all0[df_am1_all0.metric_name.isin(total_be)]
df_am1_all1=df_am1_all1.assign(metric_name='Total Business Essentials Widgets')

### order the variables and concatenate with other total metrics
df_be_all310=df_am_all1[['metric_name','level','acf2_id','employee_name', 
                            'november','december','january','february','march','april','may','june','july',
                            'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
                            'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
                            'am_cost_center','am_cost_center_name','am_cost_center_full_name',
                            'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]

df_sc_all310=df_am1_all1[['metric_name','level','acf2_id','employee_name', 
                            'november','december','january','february','march','april','may','june','july',
                            'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
                            'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
                            'am_cost_center','am_cost_center_name','am_cost_center_full_name',
                            'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]

## get only the AMSBs from the other total metrics
all_month_final320=all_month_final31[(all_month_final31.level=='AM')]

## concatenate all dataframes
all_metrics_all40=pd.concat([all_month_final320,df_be_all310,df_sc_all310],axis=0)


##groupby
df_am_all3=all_metrics_all40.groupby(by=['level','scorecard_filter','metric_name','acf2_id',
                 'am_cost_center','am_cost_center_name','am_cost_center_full_name',
                 'sm_cost_center','sm_cost_center_name','record_date'],as_index=False).aggregate({
                        'november': 'sum',
                        'december': 'sum',
                        'january': 'sum',
                        'february': 'sum',
                        'march': 'sum',
                        'april': 'sum',
                        'may': 'sum',
                        'june': 'sum',
                        'july': 'sum',
                        'august': 'sum',
                        'september': 'sum',
                        'october': 'sum'})

df_am_all3['employee_name']=df_am_all3['am_cost_center_full_name']

##CREATE THE AM QUARTER AND YTD VARIABLES
newvarfunc(df_am_all3)

## replace 0 with NaN so median only counts non-zeros
df_am_all3=df_am_all3.replace(0.0, np.nan)

## CREATE THE QTD BENCHMARK ("quarter" is a set variable (current))
## Q1
psa_am_median=df_am_all3.groupby(['metric_name'], as_index=False)['q1'].median()
##rename median
psa_am_median.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
### merge to dataframe
psa_am_all9=pd.merge(df_am_all3,psa_am_median,on=['metric_name'],
                   how="left")

## Q2
psa_am_median1=df_am_all3.groupby(['metric_name'], as_index=False)['q2'].median()
##rename median
psa_am_median1.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
### merge to dataframe
psa_am_all4=pd.merge(psa_am_all9,psa_am_median1,on=['metric_name'],
                   how="left")

## Q3
psa_am_median2=df_am_all3.groupby(['metric_name'], as_index=False)['q3'].median()
##rename median
psa_am_median2.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
### merge to dataframe
psa_am_all5=pd.merge(psa_am_all4,psa_am_median2,on=['metric_name'],
                   how="left")

## Q4
psa_am_median3=df_am_all3.groupby(['metric_name'], as_index=False)['q4'].median()
##rename median
psa_am_median3.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
### merge to dataframe
psa_am_all6=pd.merge(psa_am_all5,psa_am_median3,on=['metric_name'],
                   how="left")


## CREATE THE YTD BENCHMARK ("quarter" is a set variable (current))
psa_am_ytdmedian=df_am_all3.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
psa_am_ytdmedian.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)
### merge to dataframe
psa_am_all7=pd.merge(psa_am_all6,psa_am_ytdmedian,on=['metric_name'],
                   how="left")

## keep relevant vars
psa_am_all8=psa_am_all7[['metric_name','level','acf2_id','employee_name',
               'november','december','january','february','march','april','may','june','july',
               'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


############### group by SM

## OneTD
df_sm_all0=df_sc_all[(df_sc_all.level=='SM')]
df_sm_all1=df_sm_all0[df_sm_all0.metric_name.isin(total_onetd)]
df_sm_all1=df_sm_all1.assign(metric_name='Total OneTD Widgets')

## BE
df_sm1_all0=df_sc_all[(df_sc_all.level=='SM')]
df_sm1_all1=df_sm1_all0[df_sm1_all0.metric_name.isin(total_be)]
df_sm1_all1=df_sm1_all1.assign(metric_name='Total Business Essentials Widgets')

### order the variables and concatenate with other total metrics
df_be_all314=df_sm_all1[['metric_name','level','acf2_id','employee_name', 
                            'november','december','january','february','march','april','may','june','july',
                            'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
                            'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
                            'am_cost_center','am_cost_center_name','am_cost_center_full_name',
                            'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]

df_sc_all314=df_sm1_all1[['metric_name','level','acf2_id','employee_name', 
                            'november','december','january','february','march','april','may','june','july',
                            'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
                            'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
                            'am_cost_center','am_cost_center_name','am_cost_center_full_name',
                            'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]

## get only the AMSBs from the other total metrics
all_month_final324=all_month_final31[(all_month_final31.level=='SM')]

## concatenate all dataframes
all_metrics_all45=pd.concat([all_month_final324,df_be_all314,df_sc_all314],axis=0)

## groupby
df_sm_all3=all_metrics_all45.groupby(by=['level','scorecard_filter','metric_name','acf2_id',
                 'am_cost_center','am_cost_center_name','am_cost_center_full_name',
                 'sm_cost_center','sm_cost_center_name','record_date'],as_index=False).aggregate({
                        'november': 'sum',
                        'december': 'sum',
                        'january': 'sum',
                        'february': 'sum',
                        'march': 'sum',
                        'april': 'sum',
                        'may': 'sum',
                        'june': 'sum',
                        'july': 'sum',
                        'august': 'sum',
                        'september': 'sum',
                        'october': 'sum'})

df_sm_all3['employee_name']=df_sm_all3.sm_cost_center_name

##CREATE THE AM QUARTER AND YTD VARIABLES
newvarfunc(df_sm_all3)

## replace 0 with NaN so median only counts non-zeros
df_sm_all3=df_sm_all3.replace(0.0, np.nan)

## CREATE THE QTD BENCHMARK ("quarter" is a set variable (current))
## Q1
psa_sm_median=df_sm_all3.groupby(['metric_name'], as_index=False)['q1'].median()
##rename median
psa_sm_median.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
### merge to dataframe
psa_sm_all9=pd.merge(df_sm_all3,psa_sm_median,on=['metric_name'],
                   how="left")

## Q2
psa_sm_median1=df_sm_all3.groupby(['metric_name'], as_index=False)['q2'].median()
##rename median
psa_sm_median1.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
### merge to dataframe
psa_sm_all4=pd.merge(psa_sm_all9,psa_sm_median1,on=['metric_name'],
                   how="left")

## Q3
psa_sm_median2=df_sm_all3.groupby(['metric_name'], as_index=False)['q3'].median()
##rename median
psa_sm_median2.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
### merge to dataframe
psa_sm_all5=pd.merge(psa_sm_all4,psa_sm_median2,on=['metric_name'],
                   how="left")

## Q4
psa_sm_median3=df_sm_all3.groupby(['metric_name'], as_index=False)['q4'].median()
##rename median
psa_sm_median3.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
### merge to dataframe
psa_sm_all6=pd.merge(psa_sm_all5,psa_sm_median3,on=['metric_name'],
                   how="left")


## CREATE THE YTD BENCHMARK ("quarter" is a set variable (current))
psa_sm_ytdmedian=df_sm_all3.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
psa_sm_ytdmedian.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)
### merge to dataframe
psa_sm_all7=pd.merge(psa_sm_all6,psa_sm_ytdmedian,on=['metric_name'],
                   how="left")

## keep relevant vars
psa_sm_all8=psa_sm_all7[['metric_name','level','acf2_id','employee_name',
               'november','december','january','february','march','april','may','june','july',
               'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


######## group by National Office

## OneTD
df_nso_all0=df_sc_all[(df_sc_all.level=='National Office')]
df_nso_all1=df_nso_all0[df_nso_all0.metric_name.isin(total_onetd)]
df_nso_all1=df_nso_all1.assign(metric_name='Total OneTD Widgets')

##BE
df_nso1_all0=df_sc_all[(df_sc_all.level=='National Office')]
df_nso1_all1=df_nso1_all0[df_nso1_all0.metric_name.isin(total_be)]
df_nso1_all1=df_nso1_all1.assign(metric_name='Total Business Essentials Widgets')


### order the variables and concatenate with other total metrics
df_be_all317=df_nso1_all1[['metric_name','level','acf2_id','employee_name', 
                            'november','december','january','february','march','april','may','june','july',
                            'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
                            'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
                            'am_cost_center','am_cost_center_name','am_cost_center_full_name',
                            'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]

df_sc_all317=df_nso_all1[['metric_name','level','acf2_id','employee_name', 
                            'november','december','january','february','march','april','may','june','july',
                            'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
                            'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
                            'am_cost_center','am_cost_center_name','am_cost_center_full_name',
                            'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]

## get only the AMSBs from the other total metrics
all_month_final327=all_month_final31[(all_month_final31.level=='National Office')]

## concatenate all dataframes
all_metrics_all47=pd.concat([all_month_final327,df_be_all317,df_sc_all317],axis=0)


##groupby
df_nso_all3=all_metrics_all47.groupby(by=['level','scorecard_filter','metric_name','acf2_id',
                 'am_cost_center','am_cost_center_name','am_cost_center_full_name',
                 'sm_cost_center','sm_cost_center_name','record_date'],as_index=False).aggregate({
                        'november': 'sum',
                        'december': 'sum',
                        'january': 'sum',
                        'february': 'sum',
                        'march': 'sum',
                        'april': 'sum',
                        'may': 'sum',
                        'june': 'sum',
                        'july': 'sum',
                        'august': 'sum',
                        'september': 'sum',
                        'october': 'sum'})

df_nso_all3['employee_name']='National Office'

##CREATE THE AM QUARTER AND YTD VARIABLES
newvarfunc(df_nso_all3)

## replace 0 with NaN so median only counts non-zeros
df_nso_all3=df_nso_all3.replace(0.0, np.nan)

## CREATE THE QTD BENCHMARK ("quarter" is a set variable (current))
## Q1
psa_nso_median=df_nso_all3.groupby(['metric_name'], as_index=False)['q1'].median()
##rename median
psa_nso_median.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
### merge to dataframe
psa_nso_all9=pd.merge(df_nso_all3,psa_nso_median,on=['metric_name'],
                   how="left")

## Q2
psa_nso_median1=df_nso_all3.groupby(['metric_name'], as_index=False)['q2'].median()
##rename median
psa_nso_median1.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
### merge to dataframe
psa_nso_all4=pd.merge(psa_nso_all9,psa_nso_median1,on=['metric_name'],
                   how="left")

## Q3
psa_nso_median2=df_nso_all3.groupby(['metric_name'], as_index=False)['q3'].median()
##rename median
psa_nso_median2.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
### merge to dataframe
psa_nso_all5=pd.merge(psa_nso_all4,psa_nso_median2,on=['metric_name'],
                   how="left")

## Q4
psa_nso_median3=df_nso_all3.groupby(['metric_name'], as_index=False)['q4'].median()
##rename median
psa_nso_median3.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
### merge to dataframe
psa_nso_all6=pd.merge(psa_nso_all5,psa_nso_median3,on=['metric_name'],
                   how="left")


## CREATE THE YTD BENCHMARK ("quarter" is a set variable (current))
psa_nso_ytdmedian=df_nso_all3.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
psa_nso_ytdmedian.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)
### merge to dataframe
psa_nso_all7=pd.merge(psa_nso_all6,psa_nso_ytdmedian,on=['metric_name'],
                   how="left")

## keep relevant vars
psa_nso_all8=psa_nso_all7[['metric_name','level','acf2_id','employee_name',
               'november','december','january','february','march','april','may','june','july',
               'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]

######### CONCATENATE ALL DATAFRAMES FOR ONETD, BE, and other total metrics
df_onetd_be_all=pd.concat([psa_sc_all8, psa_am_all8,psa_sm_all8,psa_nso_all8],axis=0)


In [ ]:
## concatenate with rest of data
df_sc_all10=pd.concat([all_month_final3,df_onetd_be_all],axis=0)


In [ ]:
## check if the record_date is unique - should be
df_sc_all10.record_date.unique()

In [ ]:
df_sc_all10.metric_name.unique()

In [ ]:
#df_sc_all10[(df_sc_all10.employee_name=='GTA') & (df_sc_all10.metric_name=='AMSB Retail Referrals')]


In [ ]:
## Convert pandas dataframe to Pyspark dataframe and save in table; this dataframe contains all metrics except for LEI
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
spark.createDataFrame(df_sc_all10).write.mode("append").partitionBy("record_date").format("hive").saveAsTable("anp_cabbtdct1_final.SBB_SC_OUTPUT_ALL")


In [ ]:
## END OF PROGRAM